In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier,XGBRFClassifier, XGBRegressor, XGBRFRegressor
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from importlib import reload
from pactools.grid_search import GridSearchCVProgressBar
import sys

training_data = pd.read_csv('2024-introduction-to-data-analysis-hw-5-1/train.csv')
testing_data = pd.read_csv('2024-introduction-to-data-analysis-hw-5-1/test.csv')

training_data.dropna(inplace=True)
training_data.drop(columns=["SEASON_ID", "TEAM_ID", 'GP', 'GS', 'MIN'], inplace=True)
testing_data.drop(columns=["ID", "SEASON_ID", "TEAM_ID"], inplace=True)

label_encoder = LabelEncoder()
training_data["position"] = label_encoder.fit_transform(training_data["position"])

X = training_data.drop(columns=["position"])
y = training_data["position"]

# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
# for i in range(5, 17):
#     NUM_SELECTED_FEATURES = i
#     if 0 > NUM_SELECTED_FEATURES or NUM_SELECTED_FEATURES > len(X_train.columns):
#         raise("Invalid number of selected features")
#     rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=NUM_SELECTED_FEATURES)
#     rfe.fit(X_train, y_train)
#     print("Selected features: ", end="")
#     for i in range(X_train.shape[1]):
#         if rfe.support_[i]:
#             print(X_train.columns[i], end=", ")
#     print()
#     X_selected = rfe.transform(X_train)
#     validation_data_selected = rfe.transform(X_val)
#     testing_data_selected = testing_data[X_train.columns[rfe.support_]]
#     model = RandomForestClassifier()
#     model.fit(X_selected, y_train)
#     y_pred = model.predict(validation_data_selected)
#     print(f1_score(y_val, y_pred, average='weighted'))

NUM_SELECTED_FEATURES = 13
if 0 > NUM_SELECTED_FEATURES or NUM_SELECTED_FEATURES > len(X.columns):
    raise("Invalid number of selected features")
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=NUM_SELECTED_FEATURES)
rfe.fit(X, y)
print("Selected features: ", end="")
for i in range(X.shape[1]):
    if rfe.support_[i]:
        print(X.columns[i], end=", ")
print()
X_selected = rfe.transform(X)
testing_data_selected = testing_data[X.columns[rfe.support_]]

# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# RANDOM_FOREST_PARAMS = {
#     'n_estimators': [100, 200, 300, 400, 500],
#     'max_depth': [None, 10, 20, 30, 40, 50],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'bootstrap': [True, False]
# }
# search = RandomizedSearchCV(model, RANDOM_FOREST_PARAMS, scoring='f1_weighted', cv=skf)
# search.fit(X, y)
# print(search.best_params_)
# model = search.best_estimator_

model = RandomForestClassifier()
model.fit(X_selected, y)

y_pred = model.predict(testing_data_selected)
submission_df = pd.DataFrame()
temp = pd.read_csv('2024-introduction-to-data-analysis-hw-5-1/test.csv')
submission_df["ID"] = temp["ID"]
submission_df["position"] = label_encoder.inverse_transform(y_pred)
print(submission_df)
submission_df.to_csv("test_submission.csv", index=False)

Selected features: PLAYER_AGE, FGA, FG_PCT, FG3A, FT_PCT, OREB, DREB, REB, AST, STL, BLK, TOV, PF, 
        ID position
0        1  Forward
1        2    Guard
2        3    Guard
3        4  Forward
4        5   Center
...    ...      ...
1995  1996    Guard
1996  1997    Guard
1997  1998  Forward
1998  1999   Center
1999  2000    Guard

[2000 rows x 2 columns]


c:\Users\carot\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
